In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model
import cvxpy as cp
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

### Set random seeds


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def generate_periods(train_start, train_end, val_length_years, test_length_years, data_min, data_max):
    train_start = pd.to_datetime(train_start)
    train_end = pd.to_datetime(train_end)
    data_min = pd.to_datetime(data_min)
    data_max = pd.to_datetime(data_max)
    
    periods = {'train': [], 'val': [], 'test': []}
    
    # Adjust train_start and train_end if outside data range
    if train_start < data_min:
        print(f"Adjusting train_start from {train_start} to {data_min}")
        train_start = data_min
    if train_end > data_max:
        print(f"Adjusting train_end from {train_end} to {data_max}")
        train_end = data_max
    
    # Set validation and test periods
    val_start = train_end + timedelta(days=1)
    val_end = val_start + timedelta(days=365 * val_length_years - 1)
    test_start = val_end + timedelta(days=1)
    test_end = test_start + timedelta(days=365 * test_length_years - 1)
    
    # Adjust if test_end exceeds data_max
    if test_end > data_max:
        print(f"Adjusting test_end from {test_end} to {data_max}")
        test_end = data_max
    
    # Validate periods
    if val_start > val_end or test_start > test_end or train_end >= val_start or val_end >= test_start:
        print("Error: Invalid or overlapping periods.")
        return [], [], []
    
    periods['train'].append((train_start.strftime('%Y-%m-%d'), train_end.strftime('%Y-%m-%d')))
    periods['val'].append((val_start.strftime('%Y-%m-%d'), val_end.strftime('%Y-%m-%d')))
    periods['test'].append((test_start.strftime('%Y-%m-%d'), test_end.strftime('%Y-%m-%d')))
    
    if not periods['train']:
        print("Warning: No valid periods generated.")
    
    return periods['train'], periods['val'], periods['test']

### Load datasets


In [ ]:
try:
    index1 = pd.read_csv('AGG.csv', parse_dates=['Date'])
    index2 = pd.read_csv('VTI.csv', parse_dates=['Date'])
    index3 = pd.read_csv('VIX.csv', parse_dates=['Date'])
    index4 = pd.read_csv('DBC.csv', parse_dates=['Date'])
except FileNotFoundError as e:
    print(f"Error: Dataset file not found. Ensure index1.csv, index2.csv, index3.csv, index4.csv exist. {e}")
    raise

# Select columns and compute features
index1 = index1[['Date', 'Close', 'Open', 'High', 'Low']].set_index('Date')
index1['Volatility'] = index1['High'] - index1['Low']
index1['Momentum'] = index1['Close'] - index1['Open']
index1 = index1[['Close', 'Volatility', 'Momentum']].rename(columns={'Close': 'Index1_Close', 'Volatility': 'Index1_Volatility', 'Momentum': 'Index1_Momentum'})

index2 = index2[['Date', 'Close', 'Open', 'High', 'Low']].set_index('Date')
index2['Volatility'] = index2['High'] - index2['Low']
index2['Momentum'] = index2['Close'] - index2['Open']
index2 = index2[['Close', 'Volatility', 'Momentum']].rename(columns={'Close': 'Index2_Close', 'Volatility': 'Index2_Volatility', 'Momentum': 'Index2_Momentum'})

index3 = index3[['Date', 'Close', 'Open', 'High', 'Low']].set_index('Date')
index3['Volatility'] = index3['High'] - index3['Low']
index3['Momentum'] = index3['Close'] - index3['Open']
index3 = index3[['Close', 'Volatility', 'Momentum']].rename(columns={'Close': 'Index3_Close', 'Volatility': 'Index3_Volatility', 'Momentum': 'Index3_Momentum'})

index4 = index4[['Date', 'Close', 'Open', 'High', 'Low']].set_index('Date')
index4['Volatility'] = index4['High'] - index4['Low']
index4['Momentum'] = index4['Close'] - index4['Open']
index4 = index4[['Close', 'Volatility', 'Momentum']].rename(columns={'Close': 'Index4_Close', 'Volatility': 'Index4_Volatility', 'Momentum': 'Index4_Momentum'})

# Create daily date range
all_dates = pd.date_range(
    start=min(index1.index.min(), index2.index.min(), index3.index.min(), index4.index.min()),
    end=max(index1.index.max(), index2.index.max(), index3.index.max(), index4.index.max()),
    freq='D'
)

# Reindex and forward-fill
index1 = index1.reindex(all_dates).fillna(method='ffill')
index2 = index2.reindex(all_dates).fillna(method='ffill')
index3 = index3.reindex(all_dates).fillna(method='ffill')
index4 = index4.reindex(all_dates).fillna(method='ffill')

# Merge
data = index1.join([index2, index3, index4], how='outer').sort_index()
print(f"Data range: {data.index.min()} to {data.index.max()}")
print(f"Data shape: {data.shape}")

# Handle missing data
data = data.fillna(method='ffill').dropna()
if data.empty:
    print("Error: Merged data is empty after handling missing values.")
    raise ValueError("Empty dataset")

# Calculate logarithmic returns (for Close only)
close_data = data[['Index1_Close', 'Index2_Close', 'Index3_Close', 'Index4_Close']].rename(
    columns={'Index1_Close': 'Index1', 'Index2_Close': 'Index2', 'Index3_Close': 'Index3', 'Index4_Close': 'Index4'}
)
returns = np.log(close_data.pct_change() + 1).dropna()
if returns.empty:
    print("Error: Returns data is empty.")
    raise ValueError("Empty returns")

# Risk-free rate (least volatile index)
volatilities = returns.std() * np.sqrt(252)
min_vol_index = volatilities.idxmin()
risk_free_rate = returns[min_vol_index].mean() * 252
risk_free_rate_daily = risk_free_rate / 252
print(f"Risk-Free Rate (from {min_vol_index}): {risk_free_rate:.4f} (annualized)")

# Alternative: Fixed 1% risk-free rate
fixed_risk_free_rate = 0.01
fixed_risk_free_rate_daily = fixed_risk_free_rate / 252

# Define indices
indices = close_data.columns

## Set Custom Periods
# Training 2006–2017, validation 2018, test 2019–2025.

train_start = '2006-01-01'  # Adjusts to 2006-02-06
train_end = '2017-12-31'
val_length_years = 1  # 2018
test_length_years = 2 # 2019–2025

train_periods, val_periods, test_periods = generate_periods( train_start, train_end, val_length_years, test_length_years,data_min=data.index.min(), data_max=data.index.max()
)




In [31]:

def create_windows(features, returns=None, window_size=90):
    if len(features) < window_size + 1:
        return np.array([]), np.array([])
    X, y = [], []
    for i in range(len(features) - window_size):
        window = features[i:i+window_size]
        if len(window) == window_size:
            X.append(window)
            # Use returns for y if provided (e.g., test_returns_idx), else use features[:, 0]
            if returns is not None:
                y.append(returns[i+window_size])
            else:
                y.append(features[i+window_size, 0])  # Predict Close
    return np.array(X), np.array(y)

### Autoencoder

In [ ]:

# Define AE with input for 90 days * 3 features.

def build_autoencoder(input_dim=90*3, encoding_dim=10):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(50, activation='relu')(input_layer)
    encoded = Dense(encoding_dim, activation='relu')(encoded)
    decoded = Dense(50, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='linear')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder, encoder


### LSTM Model

In [ ]:

# Define LSTM with 20 nodes.

def build_lstm(encoding_dim=10):
    input_layer = Input(shape=(1, encoding_dim))
    lstm = LSTM(20, dropout=0.1, recurrent_dropout=0.2)(input_layer)
    output = Dense(1)(lstm)
    model = Model(input_layer, output)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss='mean_squared_error')
    return model

### MLP Model

In [ ]:

# Define MLP.

def build_mlp(encoding_dim=10):
    input_layer = Input(shape=(encoding_dim,))
    dense = Dense(30, activation='relu')(input_layer)
    dense = Dense(15, activation='relu')(dense)
    output = Dense(1)(dense)
    model = Model(input_layer, output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

### Omega Optimization

In [2]:

# Define OMEGA with δ=0.7, weight cap, volatility penalty.

def optimize_omega(predicted_returns, actual_returns, R_20, R_5, cov_matrix, delta=0.7):
    n = len(indices)
    x = cp.Variable(n)
    psi = cp.Variable()
    u = cp.Variable(10)  # T^j = 10
    l = 2  # Number of normal distributions
    
    e_hat = np.zeros((n, 10))  # Predictive errors placeholder
    
    # Volatility penalty
    portfolio_variance = cp.quad_form(x, cov_matrix)
    volatility_penalty = 0.1 * cp.sqrt(portfolio_variance)
    
    obj = (2/8) * psi + (2/8) * cp.sum(x @ predicted_returns) + (2/8) * cp.sum(x @ np.mean(e_hat, axis=1)) + \
          (1/8) * cp.sum(x @ R_20) + (1/8) * cp.sum(x @ R_5) - volatility_penalty
    
    constraints = []
    for j in range(l):
        constraints.append(delta * cp.sum(x @ predicted_returns) - ((1 - delta) / 10) * cp.sum(u) >= psi)
        for t in range(10):
            constraints.append(u[t] >= -cp.sum(x @ e_hat[:, t]))
            constraints.append(u[t] >= 0)
    constraints.append(cp.sum(x) == 1)
    constraints.append(x >= 0)
    constraints.append(x <= 0.5)
    
    prob = cp.Problem(cp.Maximize(obj), constraints)
    try:
        prob.solve(solver=cp.ECOS)
        if prob.status != 'optimal':
            return np.ones(n) / n
        return x.value
    except:
        return np.ones(n) / n

### Equal-Weighted Portfolio

In [ ]:

# Define AE+LSTM+EW.

def equal_weighted_portfolio(n):
    return np.ones(n) / n

## Training and Testing
# Train models and optimize portfolios with monthly rebalancing.

all_portfolio_returns = {'AE+LSTM+OMEGA': [], 'AE+LSTM+EW': [], 'AE+MLP+OMEGA': []}
if not train_periods:
    print("Error: No training periods generated. Cannot proceed with training.")
else:
    for i, (train_period, val_period, test_period) in enumerate(zip(train_periods, val_periods, test_periods)):
        train_start, train_end = pd.to_datetime(train_period)
        val_start, val_end = pd.to_datetime(val_period)
        test_start, test_end = pd.to_datetime(test_period)
        
        print(f"Processing period {i}: Train {train_start} to {train_end}, Val {val_start} to {val_end}, Test {test_start} to {test_end}")
        
        try:
            train_data = data.loc[train_start:train_end]
            val_data = data.loc[val_start:val_end]
            test_data = data.loc[test_start:test_end]
            test_returns = returns.loc[test_start:test_end]
            
            if train_data.empty or val_data.empty or test_data.empty:
                print(f"Skipping period {i}: Empty data for train, val, or test.")
                continue
        except KeyError as e:
            print(f"Skipping period {i}: Date range error. {e}")
            continue
        
        cov_matrix = test_returns.cov() * 252
        
        predicted_returns_lstm = {}
        predicted_returns_mlp = {}
        for idx in indices:
            train_features = train_data[[f'{idx}_Close', f'{idx}_Volatility', f'{idx}_Momentum']].values
            val_features = val_data[[f'{idx}_Close', f'{idx}_Volatility', f'{idx}_Momentum']].values
            test_features = test_data[[f'{idx}_Close', f'{idx}_Volatility', f'{idx}_Momentum']].values
            train_returns_idx = returns[idx].loc[train_start:train_end].values
            val_returns_idx = returns[idx].loc[val_start:val_end].values
            test_returns_idx = returns[idx].loc[test_start:test_end].values
            
            # Use features for X, returns for y in test
            X_train, y_train = create_windows(train_features)
            X_val, y_val = create_windows(val_features)
            X_test, y_test = create_windows(test_features, returns=test_returns_idx)
            
            if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
                print(f"Skipping index {idx}: Insufficient data for windows.")
                continue
            
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
            X_val_scaled = scaler.transform(X_val.reshape(X_val.shape[0], -1))
            X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))
            
            autoencoder, encoder = build_autoencoder()
            autoencoder.fit(X_train_scaled, X_train_scaled, epochs=50, batch_size=100, validation_data=(X_val_scaled, X_val_scaled), verbose=0)
            
            X_train_encoded = encoder.predict(X_train_scaled, verbose=0)
            X_val_encoded = encoder.predict(X_val_scaled, verbose=0)
            X_test_encoded = encoder.predict(X_test_scaled, verbose=0)
            
            X_train_lstm = X_train_encoded.reshape(X_train_encoded.shape[0], 1, X_train_encoded.shape[1])
            X_val_lstm = X_val_encoded.reshape(X_val_encoded.shape[0], 1, X_val_encoded.shape[1])
            X_test_lstm = X_test_encoded.reshape(X_test_encoded.shape[0], 1, X_test_encoded.shape[1])
            
            lstm_model = build_lstm()
            lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=100, validation_data=(X_val_lstm, y_val), verbose=0)
            predicted_lstm = lstm_model.predict(X_test_lstm, verbose=0).flatten()
            predicted_returns_lstm[idx] = predicted_lstm
            
            mlp_model = build_mlp()
            mlp_model.fit(X_train_encoded, y_train, epochs=50, batch_size=100, validation_data=(X_val_encoded, y_val), verbose=0)
            predicted_mlp = mlp_model.predict(X_test_encoded, verbose=0).flatten()
            predicted_returns_mlp[idx] = predicted_mlp
        
        if not predicted_returns_lstm or not predicted_returns_mlp:
            print(f"Skipping period {i}: No predictions generated.")
            continue
        
        R_20 = test_returns.rolling(window=20).mean().iloc[90:].values
        R_5 = test_returns.rolling(window=5).mean().iloc[90:].values
        
        for t in range(0, len(predicted_returns_lstm[indices[0]]), 21):
            actual_ret = test_returns.iloc[90+t].values
            pred_ret_lstm = np.array([predicted_returns_lstm[idx][t] for idx in indices])
            pred_ret_mlp = np.array([predicted_returns_mlp[idx][t] for idx in indices])
            
            weights_omega = optimize_omega(pred_ret_lstm, actual_ret, R_20[t], R_5[t], cov_matrix)
            daily_return_omega = np.sum(actual_ret * weights_omega)
            all_portfolio_returns['AE+LSTM+OMEGA'].append(daily_return_omega)
            
            weights_ew = equal_weighted_portfolio(len(indices))
            daily_return_ew = np.sum(actual_ret * weights_ew)
            all_portfolio_returns['AE+LSTM+EW'].append(daily_return_ew)
            
            weights_mlp = optimize_omega(pred_ret_mlp, actual_ret, R_20[t], R_5[t], cov_matrix)
            daily_return_mlp = np.sum(actual_ret * weights_mlp)
            all_portfolio_returns['AE+MLP+OMEGA'].append(daily_return_mlp)


Processing period 0: Train 2006-02-06 00:00:00 to 2017-12-31 00:00:00, Val 2018-01-01 00:00:00 to 2018-12-31 00:00:00, Test 2019-01-01 00:00:00 to 2025-05-16 00:00:00


### Performance Metrics


In [ ]:

# Calculate Sharpe ratios with both risk-free rates.

results = {}
for model in all_portfolio_returns:
    portfolio_returns = np.array(all_portfolio_returns[model])
    if len(portfolio_returns) == 0:
        print(f"Warning: No returns for {model}. Metrics set to zero.")
        results[model] = {
            'Sharpe Ratio (Min-Vol)': 0.0,
            'Sharpe Ratio (Fixed 1%)': 0.0,
            'Sharpe Ratio (Zero)': 0.0,
            'Annual Return': 0.0,
            'Total Return': 0.0
        }
    else:
        mean_return = np.mean(portfolio_returns) * 252
        std_return = np.std(portfolio_returns) * np.sqrt(252)
        sharpe_min_vol = (mean_return - risk_free_rate) / std_return if std_return != 0 else 0.0
        sharpe_fixed = (mean_return - fixed_risk_free_rate) / std_return if std_return != 0 else 0.0
        sharpe_zero = mean_return / std_return if std_return != 0 else 0.0
        total_return = np.prod(1 + portfolio_returns) - 1
        results[model] = {
            'Sharpe Ratio (Min-Vol)': sharpe_min_vol,
            'Sharpe Ratio (Fixed 1%)': sharpe_fixed,
            'Sharpe Ratio (Zero)': sharpe_zero,
            'Annual Return': mean_return,
            'Total Return': total_return
        }

for model, metrics in results.items():
    print(f"\n{model}:")
    print(f"Sharpe Ratio (Min-Vol Index): {metrics['Sharpe Ratio (Min-Vol)']:.4f}")
    print(f"Sharpe Ratio (Fixed 1%): {metrics['Sharpe Ratio (Fixed 1%)']:.4f}")
    print(f"Sharpe Ratio (Zero Risk-Free): {metrics['Sharpe Ratio (Zero)']:.4f}")
    print(f"Annual Return: {metrics['Annual Return']:.4f}")
    print(f"Total Return: {metrics['Total Return']:.4f}")



AE+LSTM+OMEGA:
Sharpe Ratio (Min-Vol Index): 5.8100
Sharpe Ratio (Fixed 1%): 5.8432
Sharpe Ratio (Zero Risk-Free): 5.8750
Annual Return: 1.8456
Total Return: 1.1396

AE+LSTM+EW:
Sharpe Ratio (Min-Vol Index): 5.8100
Sharpe Ratio (Fixed 1%): 5.8432
Sharpe Ratio (Zero Risk-Free): 5.8750
Annual Return: 1.8456
Total Return: 1.1396

AE+MLP+OMEGA:
Sharpe Ratio (Min-Vol Index): 5.8100
Sharpe Ratio (Fixed 1%): 5.8432
Sharpe Ratio (Zero Risk-Free): 5.8750
Annual Return: 1.8456
Total Return: 1.1396
